In [1]:
import os
import urllib.request
import numpy as np
from tensorflow import keras
import tensorflow as tf
from livelossplot import PlotLossesKerasTF
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, BatchNormalization, Dropout
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D, Activation
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
from livelossplot import PlotLossesKerasTF
from PIL import Image
from PIL import ImageFile
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

ImageFile.LOAD_TRUNCATED_IMAGES = True
%matplotlib inline
%config Completer.use_jedi = False

In [2]:
'''
In our dataset there is only single data we have to devide it into Train and validation
'''
'''import os
import random
import shutil

# Set the path to your dataset directory
dataset_dir = "./dataset/"

# Set the ratio of data to be used for training (e.g., 80%)
train_ratio = 0.8

# Create the "dataset" folder if it doesn't exist
dataset_folder = os.path.join(dataset_dir, "dataset")
os.makedirs(dataset_folder, exist_ok=True)

# Check if the "Train" and "Valid" folders already exist
train_dir = os.path.join(dataset_folder, "Train")
valid_dir = os.path.join(dataset_folder, "Valid")
if os.path.exists(train_dir) and os.path.exists(valid_dir):
    print("Train and Valid folders already exist. Skipping the split operation.")
else:
    # Create directories for the train and validation sets
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(valid_dir, exist_ok=True)

    # Get a list of all folders in the dataset directory
    all_folders = os.listdir(dataset_dir)
    # Remove the "dataset" folder from the list
    all_folders.remove("dataset")

    # Iterate through each folder in the dataset directory
    for folder_name in all_folders:
        folder_path = os.path.join(dataset_dir, folder_name)

        # Create subdirectories in the train and validation sets
        train_folder = os.path.join(train_dir, folder_name)
        valid_folder = os.path.join(valid_dir, folder_name)
        os.makedirs(train_folder, exist_ok=True)
        os.makedirs(valid_folder, exist_ok=True)

        # Get a list of image files in the current folder
        image_files = os.listdir(folder_path)

        # Shuffle the image files randomly
        random.shuffle(image_files)

        # Determine the split point based on the train_ratio
        split_point = int(train_ratio * len(image_files))

        # Split the image files into train and validation sets
        train_images = image_files[:split_point]
        valid_images = image_files[split_point:]

        # Move the train images to the train folder
        for image in train_images:
            src_path = os.path.join(folder_path, image)
            dst_path = os.path.join(train_folder, image)
            shutil.move(src_path, dst_path)

        # Move the validation images to the validation folder
        for image in valid_images:
            src_path = os.path.join(folder_path, image)
            dst_path = os.path.join(valid_folder, image)
            shutil.move(src_path, dst_path)
'''

'import os\nimport random\nimport shutil\n\n# Set the path to your dataset directory\ndataset_dir = "./dataset/"\n\n# Set the ratio of data to be used for training (e.g., 80%)\ntrain_ratio = 0.8\n\n# Create the "dataset" folder if it doesn\'t exist\ndataset_folder = os.path.join(dataset_dir, "dataset")\nos.makedirs(dataset_folder, exist_ok=True)\n\n# Check if the "Train" and "Valid" folders already exist\ntrain_dir = os.path.join(dataset_folder, "Train")\nvalid_dir = os.path.join(dataset_folder, "Valid")\nif os.path.exists(train_dir) and os.path.exists(valid_dir):\n    print("Train and Valid folders already exist. Skipping the split operation.")\nelse:\n    # Create directories for the train and validation sets\n    os.makedirs(train_dir, exist_ok=True)\n    os.makedirs(valid_dir, exist_ok=True)\n\n    # Get a list of all folders in the dataset directory\n    all_folders = os.listdir(dataset_dir)\n    # Remove the "dataset" folder from the list\n    all_folders.remove("dataset")\n\n   

In [3]:
basedir = "./dataset/"# here below the train and validation data

In [4]:
# Remove corrupted images
def remove_corrupted_images(path):
    for folder in os.listdir(path):
        folder_path = os.path.join(path, folder)
        if os.path.isdir(folder_path):
            for filename in os.listdir(folder_path):
                try:
                    img = Image.open(os.path.join(folder_path, filename))
                    img.verify()
                except (IOError, SyntaxError) as e:
                    print('Bad file:', filename)
                    os.remove(os.path.join(folder_path, filename))

In [5]:
yoga_train_path = os.path.join("Train") # root for training
yoga_test_path = os.path.join("Valid") # root for testing

traindir = os.path.join(basedir,yoga_train_path)
validdir = os.path.join(basedir,yoga_test_path)

In [6]:
remove_corrupted_images(traindir)
remove_corrupted_images(validdir)

Image Data Generator

In [7]:
# Enable mixed precision policy
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# Data augmentation and preprocessing
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
valid_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


In [8]:
# Parameters
batch_size = 32
image_size = (224, 224)

# Load data from directory
train_generator = train_datagen.flow_from_directory(
    directory=traindir,
    target_size=image_size,
    batch_size=batch_size,
    shuffle=True,
    color_mode="rgb",
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    directory=validdir,
    target_size=image_size,
    batch_size=batch_size,
    shuffle=False,
    color_mode="rgb",
    class_mode='categorical'
)

Found 2733 images belonging to 85 classes.
Found 730 images belonging to 85 classes.


In [9]:
'''# Create the directory if it doesn't exist
directory = './pretrained_models'
if not os.path.exists(directory):
    os.makedirs(directory)

url = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.7/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5'
file_path = os.path.join(directory, 'inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5')

urllib.request.urlretrieve(url, file_path)'''

"# Create the directory if it doesn't exist\ndirectory = './pretrained_models'\nif not os.path.exists(directory):\n    os.makedirs(directory)\n\nurl = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.7/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5'\nfile_path = os.path.join(directory, 'inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5')\n\nurllib.request.urlretrieve(url, file_path)"

In [10]:
# Create InceptionResNetV2 base model
base_model = InceptionResNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(image_size[0], image_size[1], 3)
)

In [11]:
output_neurons = 107

# Add custom classification layers on top of base model
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)  # Increase number of neurons
x = BatchNormalization()(x)  # Add batch normalization
x = Dropout(0.2)(x)
x = Dense(128, activation='relu')(x)  # Additional dense layer
x = BatchNormalization()(x)  # Add batch normalization
x = Dropout(0.2)(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = keras.Model(inputs=base_model.input, outputs=predictions)

# Fine-tune the top layers
for layer in model.layers[:-10]:  # Unfreeze more top layers for fine-tuning
    layer.trainable = False

# Adjust learning rate and optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)  # Use SGD optimizer with different learning rate and momentum
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [12]:
# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_lr=0.0001, mode='auto')  # Adjust learning rate dynamically
checkpoint = ModelCheckpoint("./checkpoints_models/pose_classification_model_weights2.h5", monitor='val_accuracy',
                             save_weights_only=True, mode='max', verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

class ConvolutionCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') >= 0.97 and logs.get('val_accuracy') >= 0.92:
            print("Reached greater than 97.0% accuracy, so cancelling training!")
            self.model.stop_training = True

convolutionCallback = ConvolutionCallback()

callbacks = [checkpoint, reduce_lr, convolutionCallback, early_stopping]

In [13]:
# Train the model
history = model.fit(
    train_generator,
    epochs=12,
    validation_data=valid_generator,
    callbacks=callbacks
)

Epoch 1/12
 7/86 [=>............................] - ETA: 37:07 - loss: 4.8878 - accuracy: 0.0312      

d:\python projects\pose_estimation\venv\lib\site-packages\PIL\Image.py:992: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


86/86 [==============================] - ETA: 0s - loss: 3.7289 - accuracy: 0.1603   

Saving Model

In [14]:
yoga_model_save_path = "./final_models/Yoga_model_1.h5"
model.save(yoga_model_save_path)

In [15]:
loaded_model = tf.keras.models.load_model('./final_models/Yoga_model_1.h5')

In [17]:
model.evaluate(valid_generator)

23/23 [==============================] - 546s 24s/step - loss: 0.8978 - accuracy: 0.7356

[0.8978368639945984, 0.7356164455413818]

In [22]:
# Define the label names
yoga_labels = {i: folder_name for i, folder_name in enumerate(sorted(os.listdir(traindir)))}

{0: 'adho mukha svanasana', 1: 'adho mukha vriksasana', 2: 'agnistambhasana', 3: 'ananda balasana', 4: 'anantasana', 5: 'anjaneyasana', 6: 'ardha bhekasana', 7: 'ardha chandrasana', 8: 'ardha matsyendrasana', 9: 'ardha pincha mayurasana', 10: 'ardha uttanasana', 11: 'ashtanga namaskara', 12: 'astavakrasana', 13: 'baddha konasana', 14: 'bakasana', 15: 'balasana', 16: 'bhairavasana', 17: 'bharadvajasana i', 18: 'bhekasana', 19: 'bhujangasana', 20: 'bhujapidasana', 21: 'bitilasana', 22: 'camatkarasana', 23: 'chaturanga dandasana', 24: 'dandasana', 25: 'dhanurasana', 26: 'durvasasana', 27: 'dwi pada viparita dandasana', 28: 'eka pada koundinyanasana i', 29: 'eka pada koundinyanasana ii', 30: 'garbha pindasana', 31: 'garudasana', 32: 'gomukhasana', 33: 'halasana', 34: 'hanumanasana', 35: 'janu sirsasana', 36: 'kapotasana', 37: 'krounchasana', 38: 'kurmasana', 39: 'lolasana', 40: 'makara adho mukha svanasana', 41: 'malasana', 42: 'marjaryasana', 43: 'matsyasana', 44: 'mayurasana', 45: 'natar

In [50]:
path = input("Enter Image Name (from 1-8) : ")
path = r"D:/python projects/pose_Estimation/yoga_pose/2d_yoga_full/finaltest_img/" + path + ".png"


def predict_image(image_path, model, target_size=(224, 224), top_k=5):
    img = keras.preprocessing.image.load_img(image_path, target_size=target_size)
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize the image

    prediction = model.predict(img_array)[0]
    class_indices = np.argsort(prediction)[::-1][:top_k]
    class_probabilities = prediction[class_indices]

    class_names = [yoga_labels[idx] for idx in class_indices]

    results = []
    for name, prob in zip(class_names, class_probabilities):
        result = f"{name}: {prob*100:.2f}"
        results.append(result)

    return results

predictions = predict_image(path, loaded_model)

for class_name in predictions:
    print(f"Class: {class_name}")

1/1 [==============================] - 1s 1s/step
Class: pincha mayurasana: 53.03
Class: adho mukha vriksasana: 39.72
Class: salamba sirsasana: 4.03
Class: salamba sarvangasana: 1.09
Class: eka pada koundinyanasana ii: 0.53


In [51]:
def predict_image(image_path, model, target_size=(224, 224)):
    img = keras.preprocessing.image.load_img(image_path, target_size=target_size)
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize the image

    prediction = model.predict(img_array)
    class_index = np.argmax(prediction)
    class_name = yoga_labels[class_index]
    return f'Predicted Yoga Pose is : {class_name}'
predict_image(path,loaded_model)

1/1 [==============================] - 1s 1s/step


'Predicted Yoga Pose is : pincha mayurasana'